In [19]:
# Prepare data

import pandas as pd
pd.set_option("display.max_columns", None)

data_url = "dataset/loadopt.csv"
dataset = pd.read_csv(data_url)

filter = (dataset['Port Code'] == 'KV-KUL-1') 
port_code = dataset.loc[filter]

port_code['Delivered Date & Time'] = pd.to_datetime(port_code['Delivered Date & Time'])
dt2 = port_code.groupby(port_code['Delivered Date & Time'].dt.strftime('%d_%m_%Y'))['Tracking Number'].count().sort_values(ascending=[False])

filter = (port_code['Delivered Date & Time'].dt.strftime('%d_%m_%Y') == dt2.index[0]) 
dated = port_code.loc[filter]

data = dated[[ 'Actual Weight', 'Actual Width', 'Actual Height', 'Actual Length']]

data.to_csv("dataset/examples.csv", index=False)

In [20]:
# Load the data

import pandas as pd
pd.set_option("display.max_columns", None)

data_url = "dataset/examples.csv"
dataset = pd.read_csv(data_url)

display(dataset)

,Actual Weight,Actual Width,Actual Height,Actual Length
0,5.700,45.0,1.0,81.0
1,19.000,38.0,39.0,85.0
2,20.000,5.0,8.0,6.0
3,36.000,45.0,95.0,48.0
4,10.000,15.0,15.0,82.0
...,...,...,...,...
362,177.000,83.0,88.0,32.0
363,8.795,50.0,21.0,61.0
364,45.000,36.0,42.0,77.0
365,26.000,56.0,55.0,56.0


In [21]:
# Get the volume of items

dataset['Volume'] = dataset['Actual Weight'] * dataset['Actual Height'] * dataset['Actual Length']

display(dataset)

,Actual Weight,Actual Width,Actual Height,Actual Length,Volume
0,5.700,45.0,1.0,81.0,461.700
1,19.000,38.0,39.0,85.0,62985.000
2,20.000,5.0,8.0,6.0,960.000
3,36.000,45.0,95.0,48.0,164160.000
4,10.000,15.0,15.0,82.0,12300.000
...,...,...,...,...,...
362,177.000,83.0,88.0,32.0,498432.000
363,8.795,50.0,21.0,61.0,11266.395
364,45.000,36.0,42.0,77.0,145530.000
365,26.000,56.0,55.0,56.0,80080.000


In [22]:
# Calculate the total weight and total volume of all items

total_volume = dataset['Volume'].sum()
total_weight = dataset['Actual Weight'].sum()

print('Total Volume: ', total_volume, 'cm3')
print('Total Weight: ', total_weight, ' kg')

Total Volume:  46254303.3265 cm3
Total Weight:  9853.69  kg


In [23]:
# Define the lorries

FEET_TO_CM = 30.48
lorries = [
    {
        "code": "4x4",
        "description": "4x4 Pickup",
        "length": 4,
        "height": 3,
        "width": 3.5,
        "max_weight": 500
    },
    {
        "code": "VAN",
        "description": "Van",
        "length": 8,
        "height": 3,
        "width": 3.5,
        "max_weight": 500
    }, 
    {
        "code": "LORRY-S",
        "description": "1-tonne lorry",
        "length": 10,
        "height": 5,
        "width": 5,
        "max_weight": 1000
    },
    {
        "code": "LORRY-M",
        "description": "3-tonne lorry",
        "length": 14,
        "height": 7.2,
        "width": 7,
        "max_weight": 3000
    },
    {
        "code": "LORRY-L",
        "description": "5-tonne lorry",
        "length": 17,
        "height": 7.2,
        "width": 7,
        "max_weight": 5000
    }    
]
for lorry in lorries:
    volume = round((lorry['length'] * FEET_TO_CM) * (lorry['height'] * FEET_TO_CM) * (lorry['width'] * FEET_TO_CM), 2)
    lorry['max_volume'] = volume
    
display(lorries)

[{'code': '4x4',
  'description': '4x4 Pickup',
  'length': 4,
  'height': 3,
  'width': 3.5,
  'max_weight': 500,
  'max_volume': 1189307.56},
 {'code': 'VAN',
  'description': 'Van',
  'length': 8,
  'height': 3,
  'width': 3.5,
  'max_weight': 500,
  'max_volume': 2378615.11},
 {'code': 'LORRY-S',
  'description': '1-tonne lorry',
  'length': 10,
  'height': 5,
  'width': 5,
  'max_weight': 1000,
  'max_volume': 7079211.65},
 {'code': 'LORRY-M',
  'description': '3-tonne lorry',
  'length': 14,
  'height': 7.2,
  'width': 7,
  'max_weight': 3000,
  'max_volume': 19980366.96},
 {'code': 'LORRY-L',
  'description': '5-tonne lorry',
  'length': 17,
  'height': 7.2,
  'width': 7,
  'max_weight': 5000,
  'max_volume': 24261874.16}]

In [24]:
# main process

from ortools.linear_solver import pywraplp
import warnings
warnings.filterwarnings('ignore')


# create data model for knapsack problem 
# paramter optimize are data to be packing into the available vehicle in totalLorry
def create_data_model(optimize, totalLorry):
    """Create the data for the example."""
    data = {}
    weights = optimize['Actual Weight'].to_list()
    volumes = optimize['Volume'].to_list()
    
    data['weights'] = weights
    data['volumes'] = volumes
    
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    
    max_volumes = []
    max_weights = []
    truck_types = []
    
    # reserve totalLorry data to be starting from small vehicle first
    totalLorry.reverse()

    # resgister max_weight and max_volume for each available vehicle
    for tL in totalLorry:
        for i in range(tL['number']):
            max_volumes.append(tL['max_volume'])
            max_weights.append(tL['max_weight'])
            truck_types.append(tL['code'])
    
    data['max_volume'] = max_volumes 
    data['max_weight'] = max_weights 
    data['truck_types'] = truck_types
    
    data['trucks'] = list(range(len(data['max_volume'])))
    
    return data


In [27]:
# ===============================
# ==== Get Load Optimization ====
# ===============================

totalLorry = [{'code': 'LORRY-L',
  'number': 1,
  'max_weight': 5000,
  'max_volume': 24261874.16},
 {'code': 'LORRY-M',
  'number': 2,
  'max_weight': 3000,
  'max_volume': 19980366.96},
 {'code': 'LORRY-S',
  'number': 3,
  'max_weight': 1000,
  'max_volume': 7079211.65},
 {'code': 'VAN', 'number': 3, 'max_weight': 500, 'max_volume': 2378615.11},
 {'code': '4x4', 'number': 6, 'max_weight': 500, 'max_volume': 1189307.56}]



In [28]:
data = create_data_model(dataset, totalLorry)

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['trucks']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['trucks']) <= 1)

# The amount packed in each bin cannot exceed its max weight.
for j in data['trucks']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i]
            for i in data['items']) <= data['max_weight'][j])

# The amount packed in each bin cannot exceed its max volume.
for j in data['trucks']:
    solver.Add(
        sum(x[(i, j)] * data['volumes'][i]
            for i in data['items']) <= data['max_volume'][j])


# Add objectives
objective = solver.Objective()

for i in data['items']:
    for j in data['trucks']:
        objective.SetCoefficient(x[(i, j)], data['volumes'][i])
objective.SetMaximization()


status = solver.Solve()


In [29]:
_totalLeftVolume = 0
_totalLeftWeight = 0
if status == pywraplp.Solver.OPTIMAL:
    assign = []
    total_weight = 0
    total_items = 0
    print('Total Lorry: ')
    display(totalLorry)
    print()
    print('Total Items:', len(dataset))
    print()
    for j in data['trucks']:
        bin_weight = 0
        bin_volume = 0
        print('Truck ', j, '[', data['truck_types'][j] ,'] - max_weight:[', "{:,.2f}".format(data['max_weight'][j]), '] - max volume:[', "{:,.2f}".format(data['max_volume'][j]), ']' )
        for i in data['items']:
            if x[i, j].solution_value() > 0:
                assign.append(i)
                total_items += 1
                print('Item', i, '- weight:', data['weights'][i],
                      ' volumes:', data['volumes'][i])
                bin_weight += data['weights'][i]
                bin_volume += data['volumes'][i]

        print('Packed truck volume:', "{:,.2f}".format(bin_volume))
        print('Packed truck weight:', "{:,.2f}".format(bin_weight))
        print()

        if (bin_volume > 0) & (bin_weight > 0):
            leftVolume = data['max_volume'][j] - bin_volume
            leftWeight = data['max_weight'][j] - bin_weight
        else:
            leftVolume = 0
            leftWeight = 0

        print('Left Volume', "{:,.2f}".format(leftVolume))
        print('Left Weight', "{:,.2f}".format(leftWeight))
        print()
        print()

        total_weight += bin_weight
        _totalLeftVolume += leftVolume
        _totalLeftWeight += leftWeight

    print('Total packed weight:', "{:,.2f}".format(total_weight))
    print('Total packed volume:', "{:,.2f}".format(objective.Value()))
    print('Total item assigned:', "{:,.0f}".format(total_items))
    print()
    print("#" * 100)
    print('Total Left Volume', "{:,.2f}".format(_totalLeftVolume))
    print('Total Left Weight', "{:,.2f}".format(_totalLeftWeight))
    print("#" * 100)

else:
    print('The problem does not have an optimal solution.')

print()

Total Lorry: 


[{'code': '4x4', 'number': 6, 'max_weight': 500, 'max_volume': 1189307.56},
 {'code': 'VAN', 'number': 3, 'max_weight': 500, 'max_volume': 2378615.11},
 {'code': 'LORRY-S',
  'number': 3,
  'max_weight': 1000,
  'max_volume': 7079211.65},
 {'code': 'LORRY-M',
  'number': 2,
  'max_weight': 3000,
  'max_volume': 19980366.96},
 {'code': 'LORRY-L',
  'number': 1,
  'max_weight': 5000,
  'max_volume': 24261874.16}]


Total Items: 367

Truck  0 [ 4x4 ] - max_weight:[ 500.00 ] - max volume:[ 1,189,307.56 ]
Item 0 - weight: 5.7  volumes: 461.7
Item 7 - weight: 13.9  volumes: 81871.0
Item 9 - weight: 2.5  volumes: 3375.0
Item 12 - weight: 6.8  volumes: 3046.4
Item 14 - weight: 25.0  volumes: 6000.0
Item 15 - weight: 1.35  volumes: 1012.5
Item 16 - weight: 7.0  volumes: 5264.0
Item 17 - weight: 6.7  volumes: 6700.0
Item 18 - weight: 3.0  volumes: 3240.0
Item 20 - weight: 4.0  volumes: 6660.0
Item 28 - weight: 2.4  volumes: 1495.2
Item 32 - weight: 10.0  volumes: 2600.0
Item 38 - weight: 5.0  volumes: 1800.0
Item 182 - weight: 24.4  volumes: 47630.508
Item 183 - weight: 13.0  volumes: 48633.0
Item 266 - weight: 0.3  volumes: 72.0
Item 285 - weight: 0.1  volumes: 48.0
Item 362 - weight: 177.0  volumes: 498432.0
Item 363 - weight: 8.795  volumes: 11266.395
Item 364 - weight: 45.0  volumes: 145530.0
Item 366 - weight: 51.0  volumes: 314160.0
Packed truck volume: 1,189,297.70
Packed truck weight: 412.94

Le

Item 251 - weight: 40.0  volumes: 112800.0
Item 252 - weight: 11.4  volumes: 53283.600000000006
Item 253 - weight: 8.0  volumes: 8320.0
Item 254 - weight: 40.0  volumes: 235600.0
Item 255 - weight: 43.0  volumes: 202272.0
Item 256 - weight: 5.0  volumes: 4000.0
Item 257 - weight: 34.5  volumes: 161287.5
Item 258 - weight: 34.5  volumes: 161287.5
Item 259 - weight: 25.0  volumes: 85250.0
Item 260 - weight: 40.0  volumes: 223560.0
Item 261 - weight: 18.0  volumes: 74700.0
Item 262 - weight: 10.0  volumes: 16000.0
Item 263 - weight: 23.5  volumes: 37224.0
Item 264 - weight: 10.0  volumes: 10270.0
Item 265 - weight: 8.0  volumes: 23556.0
Item 267 - weight: 6.0  volumes: 25896.0
Item 269 - weight: 50.0  volumes: 178750.0
Item 270 - weight: 60.0  volumes: 270000.0
Item 271 - weight: 20.0  volumes: 76260.0
Item 272 - weight: 47.0  volumes: 108570.0
Item 274 - weight: 41.0  volumes: 273552.0
Item 275 - weight: 8.0  volumes: 12800.0
Item 276 - weight: 24.0  volumes: 29952.0
Item 277 - weight: 1